# Overview Materi

Source: https://www.youtube.com/watch?v=LDRbO9a6XPU

Jelaskan secara singkat apa itu decision tree menurut pemahamanmu!

# Import Data & Libraries

In [2]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# Fungsi Dasar

In [4]:
# fungsi mencari apa saja unique value dari suatu kolom
def unique_vals(rows, col):
    return set([row[col] for row in rows])


# contoh penggunaan
print(unique_vals(training_data, 0))

{'Red', 'Yellow', 'Green'}


In [11]:
# fungsi Menghitung jumlah unique value dari suatu kolom
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

# contoh penggunaan
print(class_counts(training_data))

{'Apple': 2, 'Grape': 2, 'Lemon': 1}


In [12]:
# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
    return isinstance(value, (int, float))

# contoh penggunaan
print(is_numeric(3))
print(is_numeric("3"))

True
False


In [13]:
# kelas untuk merepresentasikan pertanyaan pada decision tree
class Question:

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
        self.column = column
        self.value = value

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return f"Is {header[self.column]} {condition} {str(self.value)}?"


# contoh penggunaan 1
q = Question(0, 'Green')
print(q)
print(q.match(training_data[0]))

Is color == Green?
True


In [14]:
# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

# contoh penggunaan
true, false = partition(training_data, Question(0, 'Red'))
print("True:", true)
print("False:", false)

True: [['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
False: [['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]


**apa itu gini impurity?**
<br> gini impurity berfungsi mengukur tingkat ketidakmurnian atau ketidakteraturan pada sebuah simpul (node) dalam pohon

In [15]:
# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for label in counts:
        prob = counts[label] / len(rows)
        impurity -= prob ** 2
    return impurity

# contoh penggunaan
print(gini(training_data))

0.6399999999999999


**apa itu information gain?**
<br> information gain berfungsi mengukur seberapa efektif sebuah fitur dalam memisahkan data berdasarkan kelas-kelasnya

In [18]:
# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):
     p = len(left) / (len(left) + len(right))
     return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

# contoh penggunaan
current_uncertainty = gini(training_data)
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
print(info_gain(true_rows, false_rows, current_uncertainty))

0.1399999999999999


In [25]:
# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
     best_gain = 0
     best_question = None
     current_uncertainty = gini(rows)
     n_features = len(rows[0]) - 1

     for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)

            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

     return best_gain, best_question

# contoh penggunaan
gain, question = find_best_split(training_data)
print("Best Gain:", gain)
print("Best Question:", question)

Best Gain: 0.37333333333333324
Best Question: Is diameter >= 3?


# Fungsi Decision Tree

In [39]:
# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:
   def __init__(self, rows):
        self.predictions = class_counts(rows)

In [88]:
# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:
  def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch


In [50]:
# membangun decision tree secara rekursif
def build_tree(rows):
    gain, question = find_best_split(rows)

    if gain == 0:
        return Leaf(rows)

    true_rows, false_rows = partition(rows, question)

    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)

    return Decision_Node(question, true_branch, false_branch)

In [87]:
# mencetak struktur decision tree secara rekursif dalam format teks
def print_tree(node, spacing=""):

    # base case: jika sudah mencapai leaf
     if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return

    # mencetak pertanyaan pada node saat ini
    print(spacing + str(node.question))

    # mencetak cabang true secara rekursif
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # mencetak cabang false secara rekursif
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")


# contoh penggunaan
print_tree(my_tree)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 13)

In [76]:
# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):
    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.predictions

    # menentukan apakah mengikuti cabang true atau cabang false
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    total = sum(counts.values())
    probs = {}
    for label in counts:
        probs[label] = str(int(counts[label] / total * 100)) + "%"
    return probs
# contoh penggunaan
for row in testing_data:
    print(f"Actual: {row[-1]}, Predicted: {print_leaf(classify(row, my_tree))}")

NameError: name 'my_tree' is not defined

In [79]:
# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    total = sum(counts.values())
    probs = {}
    for label in counts:
        probs[label] = str(int(counts[label] / total * 100)) + "%"
    return probs

# contoh penggunaan
for row in testing_data:
    print(f"Actual: {row[-1]}, Predicted: {print_leaf(classify(row, my_tree))}")

NameError: name 'my_tree' is not defined

# Predict Using Decision Tree

In [80]:
# menguji decision tree dengan data uji dan membandingkan hasil prediksi dengan label asli
for row in testing_data:
    pred = classify(row, my_tree)
    print(f"Data: {row} -> Predicted: {print_leaf(pred)}")

NameError: name 'my_tree' is not defined